In [1]:
# MCTS. For real this time.

In [2]:
import numpy as np
import random
import math
import sys

In [3]:
# remove index from list in O(1) without preserving order
def fast_remove(list, index):
    if index < len(list):
        list[-1], list[index] = list[index], list[-1]
        list.pop() # O(1) to pop last (default)
    return list

In [4]:
def mand(bools): # assumes list of bools
    return not False in bools

assert mand([True])
assert not mand([True, False])
assert not mand([False])

In [90]:
class TicTacToeGame():
    def __init__(self, state=None, playing=1):
        # state logic:
        # we use playing == None to represent a finished game
        # and we represent draws with winner == None when game is finished (i.e. playing == None)
        if state is not None:
            assert type(state) == np.ndarray, "state must be ndarray"
            assert state.shape == (3, 3), "state.shape must be (3, 3)"
        self.state = state if state is not None else np.zeros((3, 3), dtype=int)
        self.playing = playing
        self.winner = None
        self._open_indices = self._get_open_indices()
    
    @property
    def opponent(self):
        return self.playing ^ 3

    def __repr__(self):
        #return f'\n{str(self.state)}\n' + (f'current player: {self.playing}' if self.playing is not None else (f'winner: {self.winner}' if self.winner is not None else 'draw'))
        return f'\n{str(self.state)}\n' + (f'current player: {self.playing}' if self.playing is not None else (f'winner: {self.winner}' if self.winner is not None else 'draw'))

    def play(self, pos):
        assert pos in self._open_indices, f'Move {pos} is not open.'
        x, y = pos
        assert self.state[y][x] == 0, 'Nonzero index should not be open'
        fast_remove(self._open_indices, self._open_indices.index(pos))

        self.state[y][x] = self.playing
        self._update_winner(x, y) # update winner

        if self._open_indices == []:
            self.playing = None
        
        if self.playing is not None:
            self.playing = self.opponent

        return self

    def _update_winner(self, x, y):
        def set_winner():
            self.winner = self.playing
            self.playing = None
            self._open_indices = []

        row = [(i, y) for i in range(0, 3)]
        col = [(x, i) for i in range(0, 3)]
        
        first_diag = list(zip(range(0, 3), range(0, 3)))
        second_diag = list(zip(range(2, -1, -1), range(0, 3)))

        relevant_diags = [d for d in [first_diag, second_diag] if (x, y) in d]
        
        to_check = [row, col] + relevant_diags
        
        for iset in to_check:
            vals = [self.state[iy][ix] == self.playing for (ix, iy) in iset]
            if mand(vals):
                set_winner()
                return

    def random_finish(self):
        while (self.winner == None) and self._open_indices:
            move = random.randint(0, len(self._open_indices)-1)
            self.play(self._open_indices[move])

    def _get_open_indices(self):
        return [(x, y)
                for x in range(3)
                for y in range(3)
                if self.state[y][x] == 0]

    def copy(self):
        tmp = TicTacToeGame(state=self.state.copy())
        tmp.playing = self.playing
        return tmp

In [91]:
# _update_winner tests

def as_eq(check, expect):
    assert check == expect, f'expected {expect}, got {check}'

g = TicTacToeGame(state=np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
]))
g._update_winner(0, 0)
as_eq(g.winner, 1)

g = TicTacToeGame(state=np.array([
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0]
]))
g._update_winner(0, 0)
as_eq(g.winner, 1)

g = TicTacToeGame(state=np.array([
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0]
]))
g._update_winner(1, 2)
as_eq(g.winner, 1)


g = TicTacToeGame(state=np.array([
    [0, 2, 0],
    [0, 2, 0],
    [0, 2, 0]
]))
g.playing = 2
g._update_winner(1, 0)
as_eq(g.winner, 2)

g = TicTacToeGame(state=np.array([
    [1, 2, 1],
    [1, 2, 1],
    [1, 2, 1]
]))
g.playing = 2
g._update_winner(1, 2)
as_eq(g.winner, 2)

g = TicTacToeGame(state=np.array([
    [0, 0, 0],
    [2, 2, 2],
    [0, 0, 0]
]))
g.playing = 2
g._update_winner(1, 1)
as_eq(g.winner, 2)
    

# fast_remove tests
as_eq(fast_remove([], 0), [])
as_eq(fast_remove([1], 0), [])
as_eq(fast_remove([1, 2], 0), [2])
as_eq(fast_remove([1, 2], 1), [1])
as_eq(fast_remove([1, 2], 2), [1, 2])
as_eq(fast_remove([1, 2, 3], 0), [3, 2])
as_eq(fast_remove([1, 2, 3], 1), [1, 3])

In [92]:
g = TicTacToeGame()
#print(g.state)

g.play((0, 0))
g.play((1, 1))

g1 = g.copy()
g1.random_finish()

g, g1

(
 [[1 0 0]
  [0 2 0]
  [0 0 0]]
 current player: 1,
 
 [[1 2 1]
  [2 2 2]
  [1 1 0]]
 winner: 2)

In [93]:
br = '\n'
comma = ','
root2 = math.sqrt(2)

In [213]:
class MCTSnode():
    def __init__(self, game, player=None, move=None):
        self.total = 0
        self.visits = 0
        self.C = root2
        self.game = game
        self.player = player if player else game.opponent
        self.move = move
        self.children = None

    def __repr__(self):
        return f'\n{str(self.game.state).replace(br, comma)}, {self.player=}, {self.move=}, {self.total=}, {self.visits=} {self.children=}'

    def _print_direct_children(self):
        for child in self.children:
            print((child.game.state, child.total, child.visits))

    def UCB1(self, parent_visits):
        if self.visits == 0:
            return sys.float_info.max
        else:
            return (self.total / self.visits) + self.C * math.sqrt(math.log(parent_visits) / self.visits)

    # Score function for given game
    def score(self, game):
        assert game.playing == None, "Cannot score incomplete game."
        if game.winner == self.player:
            return 1
        elif game.winner == None: # draw
            return 0
        else:
            return -1

    def simulate(self):
        simulation = self.game.copy()
        if simulation.winner == None:
            simulation.random_finish()
        return self.score(simulation)
        # maybe "random finish" should be in an MCTS subclass or in a wrapper
        # class on the game class ?
        # if we assume it should be in a wrapper class on game class,
        # nothing actually has to change from the perspective of MCTSnode...

    def rollout(self):
        score = self.simulate()
        self.total += score
        self.visits += 1
        return score
        # "rollout" of a terminal node just returns the winner.

    @property
    def opponent(self):
        return self.player ^ 3 # is it a problem that we have an identical function
        # here and in the game class?

    def add_children(self):
        assert self.game._open_indices
        assert self.game.winner == None
        
        self.children = [
            MCTSnode(self.game.copy().play(move), player=self.opponent, move=move) for move in self.game._open_indices
        ]

    def iterate_best_child(self):
        UCBs = [ (child.UCB1(self.visits), i) for (i, child) in list(enumerate(self.children)) ]
        (max_ucb, i) = max(UCBs)
        score = -self.children[i].iteration() # negative score because we're scoring opponent's move
        self.total += score
        self.visits += 1
        return score

    def iteration(self):
        if self.game.playing is None:
            score = self.score(self.game)
            self.total += score
            self.visits += 1
            return score
            
        if self.visits == 0 or self.children == []: # unvisited OR guaranteed losing node -- no children
            return self.rollout()
        if self.children is None:
            self.add_children()
            return self.iteration() # can't call iterate_best_child() directly since we don't know if add_children added anything
        else: # have children
            return self.iterate_best_child()

    def iterate(self, n=1):
        for i in range(n):
            self.iteration()
        return self

    def best_move(self):
        if self.children == None:
            return None

        best_child = self.children[0]
        best_UCB1 = best_child.UCB1(self.visits)
        for child in self.children[1:]:
            child_score = child.UCB1(self.visits)
            if child_score > best_UCB1:
                best_child = child
                best_UCB1 = child_score
    
        # for i in range(3):
        #     for j in range(3):
        #         if self.game.state[i][j] == 0 and best_child.game.state[i][j] == self.player:
        #             return (j, i) # if we store move this could be self.move

        if best_child.move:
            return best_child.move
        
        print("ERROR")
        print(self.game)
        print(self.game._open_indices)
        assert False, "Didn't find any move in best_move"




In [253]:
def AIgame(player=1, iterations=1000):
    assert player == 1 or player == 2, "Player can only be 1 or 2"

    g = TicTacToeGame()
    
    while g.playing:
        print(g)
        if g.playing == player:
            x, y = None, None
            while (x, y) == (None, None):
                _x, _y = input("Input two numbers (separated by space)").split(' ')
                try:
                    (x, y) = int(_x), int(_y)
                except ValueError:
                    "Invalid input"
            g.play((x, y))
        else:
            MCTS = MCTSnode(g.copy())
            MCTS.iterate(iterations)
            g.play(MCTS.best_move())

    print(g)


In [254]:
AIgame(iterations=10000)


[[0 0 0]
 [0 0 0]
 [0 0 0]]
current player: 1


Input two numbers (separated by space) 0 0



[[1 0 0]
 [0 0 0]
 [0 0 0]]
current player: 2

[[1 0 0]
 [0 2 0]
 [0 0 0]]
current player: 1


Input two numbers (separated by space) 2 2



[[1 0 0]
 [0 2 0]
 [0 0 1]]
current player: 2

[[1 0 0]
 [2 2 0]
 [0 0 1]]
current player: 1


Input two numbers (separated by space) 2 1



[[1 0 0]
 [2 2 1]
 [0 0 1]]
current player: 2

[[1 0 2]
 [2 2 1]
 [0 0 1]]
current player: 1


Input two numbers (separated by space) 1 2



[[1 0 2]
 [2 2 1]
 [0 1 1]]
current player: 2

[[1 0 2]
 [2 2 1]
 [2 1 1]]
winner: 2


In [252]:
# based on empirical testing, AI seems to play perfectly pretty much 100% of the time at 10000+ iters...

In [256]:
def TrueAIGame(iterations=1000):
    g = TicTacToeGame()
    while g.playing:
        print(g)
        MCTS = MCTSnode(g.copy())
        MCTS.iterate(iterations)
        g.play(MCTS.best_move())

    print(g)

In [266]:
TrueAIGame()


[[0 0 0]
 [0 0 0]
 [0 0 0]]
current player: 1

[[0 0 0]
 [1 0 0]
 [0 0 0]]
current player: 2

[[0 0 0]
 [1 2 0]
 [0 0 0]]
current player: 1

[[0 0 0]
 [1 2 0]
 [1 0 0]]
current player: 2

[[2 0 0]
 [1 2 0]
 [1 0 0]]
current player: 1

[[2 0 0]
 [1 2 0]
 [1 0 1]]
current player: 2

[[2 0 0]
 [1 2 0]
 [1 2 1]]
current player: 1

[[2 1 0]
 [1 2 0]
 [1 2 1]]
current player: 2

[[2 1 2]
 [1 2 0]
 [1 2 1]]
current player: 1

[[2 1 2]
 [1 2 1]
 [1 2 1]]
draw
